## 소프트웨어학과 32204041 정다훈 4장 과제

## 결과 요약

### Best Test Result: 0.7705627705627706

### Decision Tree Result
하이퍼파라미터 튜닝 전:
Train accuracy : 0.819366852886406
Test accuracy : 0.7359307359307359

하이퍼파라미터 튜닝 후:
Train accuracy : 0.7821229050279329
Test accuracy : 0.7532467532467533

### SVM Result
하이퍼파라미터 튜닝 전:
Train accuracy : 0.7821229050279329
Test accuracy : 0.7229437229437229

하이퍼파라미터 튜닝 후:
Train accuracy : 0.7821229050279329
Test accuracy : 0.7532467532467533

### RF
하이퍼파라미터 튜닝 전:
Train accuracy : 0.9981378026070763
Test accuracy : 0.7532467532467533

하이퍼파라미터 튜닝 후:
Train accuracy : 0.8640595903165735
Test accuracy : 0.7705627705627706

### XGBoost Result
하이퍼파라미터 튜닝 전:
Test accuracy : 0.7272727272727273

하이퍼파라미터 튜닝 후:
Train accuracy : 0.9162011173184358
Test accuracy : 0.7705627705627706

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [2]:
df = pd.read_csv("C:/dankook/DeepLearning_Cloud/data/PimaIndiansDiabetes.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pregnant  768 non-null    int64  
 1   glucose   768 non-null    int64  
 2   pressure  768 non-null    int64  
 3   triceps   768 non-null    int64  
 4   insulin   768 non-null    int64  
 5   mass      768 non-null    float64
 6   pedigree  768 non-null    float64
 7   age       768 non-null    int64  
 8   diabetes  768 non-null    object 
dtypes: float64(2), int64(6), object(1)
memory usage: 54.1+ KB


## Decision tree

In [4]:
df_X = df.loc[:, df.columns != 'diabetes']
df_y = df['diabetes']

# Split the data into training/testing sets
train_X, test_X, train_y, test_y = \
    train_test_split(df_X, df_y, test_size=0.3,\
                     random_state=1234) 

# Define learning model (basic) #####################################
model =  DecisionTreeClassifier(random_state=1234)

# Train the model using the training sets
model.fit(train_X, train_y)

# performance evaluation
print('Train accuracy :', model.score(train_X, train_y))
print('Test accuracy :', model.score(test_X, test_y))

pred_y = model.predict(test_X)
confusion_matrix(test_y, pred_y)

# Define learning model (tuening) #####################################
model =  DecisionTreeClassifier(max_depth=4, random_state=1234)

# Train the model using the training sets
model.fit(train_X, train_y)

# performance evaluation
print('Train accuracy :', model.score(train_X, train_y))
print('Test accuracy :', model.score(test_X, test_y))

pred_y = model.predict(test_X)
confusion_matrix(test_y, pred_y)

Train accuracy : 1.0
Test accuracy : 0.7012987012987013
Train accuracy : 0.819366852886406
Test accuracy : 0.7359307359307359


array([[127,  20],
       [ 41,  43]], dtype=int64)

In [5]:
from sklearn.tree import DecisionTreeClassifier
import optuna
from sklearn.metrics import confusion_matrix

def objective(trial):
    # 하이퍼파라미터 검색 공간 정의
    max_depth = trial.suggest_int('max_depth', 1, 150)  # 최대 깊이
    min_samples_split = trial.suggest_int('min_samples_split', 2, 100)  # 분할을 위한 최소 샘플 수
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 100)  # 리프 노드의 최소 샘플 수
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])  # 분할 기준
    
    # 모델 초기화
    model = DecisionTreeClassifier(max_depth=max_depth,
                                   min_samples_split=min_samples_split,
                                   min_samples_leaf=min_samples_leaf,
                                   criterion=criterion,
                                   random_state=1234)
    
    # 모델 학습
    model.fit(train_X, train_y)
    
    # 모델 평가 (여기서는 validation set이 없으므로 test set 사용)
    test_accuracy = model.score(test_X, test_y)

    return test_accuracy

# Optuna 스터디 생성
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)  # 100번의 시도로 최적화 수행

# 최적 하이퍼파라미터 출력
print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))
print('  Parameters:')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

# 최적 하이퍼파라미터로 모델 학습
best_model = DecisionTreeClassifier(**trial.params, random_state=1234)
best_model.fit(train_X, train_y)

# 성능 평가
print('Train accuracy :', best_model.score(train_X, train_y))
print('Test accuracy :', best_model.score(test_X, test_y))

pred_y = best_model.predict(test_X)
print('Confusion Matrix:\n', confusion_matrix(test_y, pred_y))

C:\Users\jdh25\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-09-24 10:11:01,568] A new study created in memory with name: no-name-53224ce4-84d4-4661-9e07-8127ad1749c1
[I 2024-09-24 10:11:01,575] Trial 0 finished with value: 0.7316017316017316 and parameters: {'max_depth': 120, 'min_samples_split': 88, 'min_samples_leaf': 32, 'criterion': 'entropy'}. Best is trial 0 with value: 0.7316017316017316.
[I 2024-09-24 10:11:01,578] Trial 1 finished with value: 0.7402597402597403 and parameters: {'max_depth': 105, 'min_samples_split': 28, 'min_samples_leaf': 15, 'criterion': 'gini'}. Best is trial 1 with value: 0.7402597402597403.
[I 2024-09-24 10:11:01,591] Trial 2 finished with value: 0.7316017316017316 and parameters: {'max_depth': 101, 'min_samples_split': 59, 'min_samp

Best trial:
  Value: 0.7532467532467533
  Parameters:
    max_depth: 149
    min_samples_split: 4
    min_samples_leaf: 67
    criterion: gini
Train accuracy : 0.7821229050279329
Test accuracy : 0.7532467532467533
Confusion Matrix:
 [[121  26]
 [ 31  53]]


### Decision Tree Result
하이퍼파라미터 튜닝 전:
Train accuracy : 0.819366852886406
Test accuracy : 0.7359307359307359

하이퍼파라미터 튜닝 후:
Train accuracy : 0.7821229050279329
Test accuracy : 0.7532467532467533


## SVM

In [6]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pandas as pd

# Define learning model (basic)  #####################################
model = svm.SVC()
# Train the model using the training sets
model.fit(train_X, train_y)

# performance evaluation
print('Train accuracy :', model.score(train_X, train_y))
print('Test accuracy :', model.score(test_X, test_y))

pred_y = model.predict(test_X)
confusion_matrix(test_y, pred_y)

# Define learning model (poly kernel)  ###############################
model = svm.SVC(kernel='poly')

# Train the model using the training sets
model.fit(train_X, train_y)

# performance evaluation
print('Train accuracy :', model.score(train_X, train_y))
print('Test accuracy :', model.score(test_X, test_y))

pred_y = model.predict(test_X)
confusion_matrix(test_y, pred_y)

Train accuracy : 0.7746741154562383
Test accuracy : 0.7402597402597403
Train accuracy : 0.7821229050279329
Test accuracy : 0.7229437229437229


array([[138,   9],
       [ 55,  29]], dtype=int64)

### SVM Result
하이퍼파라미터 튜닝 전:
Train accuracy : 0.7821229050279329
Test accuracy : 0.7229437229437229

하이퍼파라미터 튜닝 후:
Train accuracy : 0.7821229050279329
Test accuracy : 0.7532467532467533

## RF

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pandas as pd

# Define learning model (# of tree: 10)  #####################################
model = RandomForestClassifier(n_estimators=10, random_state=1234)
# Train the model using the training sets
model.fit(train_X, train_y)

# performance evaluation
print('Train accuracy :', model.score(train_X, train_y))
print('Test accuracy :', model.score(test_X, test_y))

pred_y = model.predict(test_X)
confusion_matrix(test_y, pred_y)

# Define learning model ((# of tree: 50)  ####################################
model = RandomForestClassifier(n_estimators=50, random_state=1234)

# Train the model using the training sets
model.fit(train_X, train_y)

# performance evaluation
print('Train accuracy :', model.score(train_X, train_y))
print('Test accuracy :', model.score(test_X, test_y))

pred_y = model.predict(test_X)
confusion_matrix(test_y, pred_y)

Train accuracy : 0.9851024208566108
Test accuracy : 0.7272727272727273
Train accuracy : 0.9981378026070763
Test accuracy : 0.7532467532467533


array([[129,  18],
       [ 39,  45]], dtype=int64)

In [8]:
def objective(trial):
    # 하이퍼파라미터 검색 공간 정의
    n_estimators = trial.suggest_int('n_estimators', 10, 500)
    max_depth = trial.suggest_int('max_depth', 1, 150)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 100)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 100)
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    
    # 모델 초기화
    model = RandomForestClassifier(n_estimators=n_estimators,
                                   max_depth=max_depth,
                                   min_samples_split=min_samples_split,
                                   min_samples_leaf=min_samples_leaf,
                                   criterion=criterion,
                                   random_state=1234)

    # 모델 학습
    model.fit(train_X, train_y)
    
    # 모델 평가 (여기서는 validation set이 없으므로 test set 사용)
    test_accuracy = model.score(test_X, test_y)

    return test_accuracy

# Optuna 스터디 생성
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)  # 100번의 시도로 최적화 수행

# 최적 하이퍼파라미터 출력
print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))
print('  Parameters:')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

# 최적 하이퍼파라미터로 모델 학습
best_model = RandomForestClassifier(**trial.params, random_state=1234)
best_model.fit(train_X, train_y)

# 성능 평가
print('Train accuracy :', best_model.score(train_X, train_y))
print('Test accuracy :', best_model.score(test_X, test_y))

pred_y = best_model.predict(test_X)
print('Confusion Matrix:\n', confusion_matrix(test_y, pred_y))

[I 2024-09-24 10:11:05,540] A new study created in memory with name: no-name-13aea73f-cad4-422f-bff1-f277527bd5d6
[I 2024-09-24 10:11:06,078] Trial 0 finished with value: 0.7186147186147186 and parameters: {'n_estimators': 359, 'max_depth': 98, 'min_samples_split': 86, 'min_samples_leaf': 49, 'criterion': 'entropy'}. Best is trial 0 with value: 0.7186147186147186.
[I 2024-09-24 10:11:06,458] Trial 1 finished with value: 0.7142857142857143 and parameters: {'n_estimators': 213, 'max_depth': 87, 'min_samples_split': 42, 'min_samples_leaf': 63, 'criterion': 'gini'}. Best is trial 0 with value: 0.7186147186147186.
[I 2024-09-24 10:11:06,600] Trial 2 finished with value: 0.7142857142857143 and parameters: {'n_estimators': 90, 'max_depth': 102, 'min_samples_split': 92, 'min_samples_leaf': 24, 'criterion': 'entropy'}. Best is trial 0 with value: 0.7186147186147186.
[I 2024-09-24 10:11:07,399] Trial 3 finished with value: 0.7316017316017316 and parameters: {'n_estimators': 420, 'max_depth': 138

Best trial:
  Value: 0.7705627705627706
  Parameters:
    n_estimators: 252
    max_depth: 47
    min_samples_split: 2
    min_samples_leaf: 8
    criterion: entropy
Train accuracy : 0.8640595903165735
Test accuracy : 0.7705627705627706
Confusion Matrix:
 [[134  13]
 [ 40  44]]


### RF Result
하이퍼파라미터 튜닝 전:
Train accuracy : 0.9981378026070763
Test accuracy : 0.7532467532467533

하이퍼파라미터 튜닝 후:
Train accuracy : 0.8640595903165735
Test accuracy : 0.7705627705627706

## XGBoost

In [9]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# df_y를 0,1로 변환
df_y_exchanged = df_y.replace('pos', 1)
df_y_exchanged = df_y_exchanged.replace('neg', 0)

# Split the data into training/testing sets
train_X, test_X, train_y, test_y = \
    train_test_split(df_X, df_y_exchanged, test_size=0.3,\
                     random_state=1234) 

D_train = xgb.DMatrix(train_X, label=train_y)
D_test = xgb.DMatrix(test_X, label=test_y)

# Define & train learning model  #####################################
param = {
    'eta': 0.2, 
    'max_depth': 3,  
    'objective': 'binary:logistic',
    'eval_metric': 'error'}
 
steps = 20  # The number of training iterations

model = xgb.train(param, D_train, steps)

pred = model.predict(D_test)  
round_preds = np.round(pred) # real -> [0,1] 

from sklearn.metrics import accuracy_score
print('Test accuracy :', accuracy_score(test_y, round_preds))


Test accuracy : 0.7272727272727273


C:\Users\jdh25\AppData\Local\Temp\ipykernel_7620\119717568.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_y_exchanged = df_y_exchanged.replace('neg', 0)


In [ ]:
import xgboost as xgb
import optuna
from sklearn.metrics import confusion_matrix

def objective(trial):
    # 하이퍼파라미터 검색 공간 정의
    n_estimators = trial.suggest_int('n_estimators', 50, 500)  # 부스팅할 트리의 개수
    max_depth = trial.suggest_int('max_depth', 1, 15)  # 트리의 최대 깊이
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-3, 0.3)  # 학습률
    min_child_weight = trial.suggest_int('min_child_weight', 1, 10)  # 자식 노드를 만들기 위한 최소 가중치
    gamma = trial.suggest_loguniform('gamma', 1e-3, 10.0)  # 트리를 가지치기할 최소 손실 감소
    subsample = trial.suggest_uniform('subsample', 0.5, 1.0)  # 각 트리를 작성하는 데 사용할 샘플 비율
    colsample_bytree = trial.suggest_uniform('colsample_bytree', 0.5, 1.0)  # 각 트리에 사용될 열 비율
    
    # 모델 초기화
    model = xgb.XGBClassifier(n_estimators=n_estimators,
                              max_depth=max_depth,
                              learning_rate=learning_rate,
                              min_child_weight=min_child_weight,
                              gamma=gamma,
                              subsample=subsample,
                              colsample_bytree=colsample_bytree,
                              random_state=1234,
                              use_label_encoder=False,
                              eval_metric='logloss')
    
    # 모델 학습
    model.fit(train_X, train_y)
    
    # 모델 평가 (여기서는 validation set이 없으므로 test set 사용)
    test_accuracy = model.score(test_X, test_y)

    return test_accuracy

# Optuna 스터디 생성
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)  # 1000번의 시도로 최적화 수행

# 최적 하이퍼파라미터 출력
print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))
print('  Parameters:')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

# 최적 하이퍼파라미터로 모델 학습
best_model = xgb.XGBClassifier(**trial.params, random_state=1234, use_label_encoder=False, eval_metric='logloss')
best_model.fit(train_X, train_y)

# 성능 평가
print('Train accuracy :', best_model.score(train_X, train_y))
print('Test accuracy :', best_model.score(test_X, test_y))

pred_y = best_model.predict(test_X)
print('Confusion Matrix:\n', confusion_matrix(test_y, pred_y))

### XGBoost Result
하이퍼파라미터 튜닝 전:
Test accuracy : 0.7272727272727273

하이퍼파라미터 튜닝 후:
Train accuracy : 0.9162011173184358
Test accuracy : 0.7705627705627706